In [9]:
import twitter

# initialize api instance
twitter_api = twitter.Api(consumer_key='dPgoCHhfjR5ffK1AlAKUABt72',
                        consumer_secret='gtMC9sBjlrxU5CZiGrRpbicbGD57v5OtcbKHmdHTzy29hdsaJX',
                        access_token_key='634480501-pSKdygTJs4iXCYmE17h6pUMVz7HeMgZbjtqzuQtd',
                        access_token_secret='rWaYZ2hQiMtH2A8wlSz7kA9PxMPndf1a1V7gSPxgOKkeF')

# test authentication
print(twitter_api.VerifyCredentials())

{"created_at": "Fri Jul 13 10:46:10 +0000 2012", "default_profile": true, "favourites_count": 1, "followers_count": 1, "friends_count": 6, "id": 634480501, "id_str": "634480501", "name": "White Sleep", "profile_background_color": "C0DEED", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme1/bg.png", "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme1/bg.png", "profile_image_url": "http://pbs.twimg.com/profile_images/2393717288/coolskull_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/2393717288/coolskull_normal.jpg", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "WhiteSleep13", "status": {"created_at": "Tue Feb 26 14:39:01 +0000 2019", "id": 1100404876599672832, "id_str": "1100404876599672832", "in_reply_to_screen_name": "yeashahrukh", "in_reply_to_statu

In [10]:
def buildTestSet(search_keyword):
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count = 100)
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [[status.text, None] for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

In [11]:
search_term = input("Enter a search keyword:")
testDataSet = buildTestSet(search_term)

print(testDataSet[0:4])

Enter a search keyword:corona virus
Fetched 100 tweets for the term corona virus
[['कांग्रेस विधायक की गाड़ी में मिली विदेशी शराब\n(@rohit_manas ) #Bihar\nhttps://t.co/jdEZmQkfAy', None], ['i’m just gonna say it, I hate corona virus, I hate quarantine', None], ['WHO menyatakan bahwa virus Corona (COVID-19) mungkin tidak akan pernah hilang dan penduduk Bumi harus belajar untuk… https://t.co/ozQ4vv0QUP', None], ['@GallagherCiaran Corona is like crunching a very fine, pale seashell. Virus is spring onion crisps.', None]]


In [12]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            label = 'negative'
            if(tweet[1]!=None):
                if float(tweet[1])>0:
                    label = 'positive'
            processedTweets.append((self._processTweet(tweet[0]),label))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Biswas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Biswas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import csv
tweet_data = []
with open("ct1_tweets_label.csv", 'r',encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    header = next(csv_reader)
    print("header  = ")
    print(header)
    for row in csv_reader:
        tweet_data.append([row[0],row[1].rstrip()])
        print([row[0],row[1].rstrip()])
print(len(tweet_data))

In [15]:
tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(tweet_data[0:1000])
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)


preprocessedTrainingSet = preprocessedTrainingSet[0:1000]
print(preprocessedTrainingSet[0:4])
print(len(preprocessedTrainingSet))

[(['rt', 'lot', 'would', 'actually', 'benefit', 'taking', 'good', 'look', 'introspectively', 'issues', 'women', 'gros…'], 'positive'), (['rt', 'day', '5', 'rediscovered', 'farming'], 'positive'), (['rt', 'mayorkun', 'somewhere', 'studio', 'singing', '``', 'ramona', 'body', 'dey', 'killing', 'person', 'like', 'corona', "''"], 'negative'), (['rt', 'appreciate', 'pak', 'jokowi', 'pak', 'prabowo', 'moves', 'fight', 'corona', 'never', 'die-hard', 'fan', 'hater', 'j…'], 'negative')]
1000


In [16]:
import nltk 

def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [17]:
def extract_features(tweet):
    tweet_words=set(tweet)
    features={}
    for word in word_features:
        features['contains(%s)' % word]=(word in tweet_words)
    return features 


In [18]:
# Now we can extract the features and train the classifier 
word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures=nltk.classify.apply_features(extract_features,preprocessedTrainingSet)

In [19]:
NBayesClassifier=nltk.NaiveBayesClassifier.train(trainingFeatures)

In [43]:
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

In [44]:
# get the majority vote
if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
    print("Overall Positive Sentiment")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
else: 
    print("Overall Negative Sentiment")
    print("Negative Sentiment Percentage = " + str(100*NBResultLabels.count('negative')/len(NBResultLabels)) + "%")


Overall Negative Sentiment
Negative Sentiment Percentage = 99.0%


In [24]:
# Example of calculating class probabilities
from math import sqrt
from math import pi
from math import exp

# Split the dataset by class values, returns a dictionary
def separate_by_class(dataset):
    separated = dict()
    for i in range(len(dataset)):
        vector = dataset[i]
        class_value = vector[-1]
        if (class_value not in separated):
            separated[class_value] = list()
        separated[class_value].append(vector)
    return separated
# Calculate the mean of a list of numbers
def mean(numbers):
    return sum(numbers)/float(len(numbers))

# Calculate the standard deviation of a list of numbers
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([(x-avg)**2 for x in numbers]) / float(len(numbers)-1)
    return sqrt(variance)

# Calculate the mean, stdev and count for each column in a dataset
def summarize_dataset(dataset):
    summaries = [(mean(column), stdev(column), len(column)) for column in zip(*dataset)]
    del(summaries[-1])
    return summaries

# Split dataset by class then calculate statistics for each row
def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    summaries = dict()
    for class_value, rows in separated.items():
        summaries[class_value] = summarize_dataset(rows)
    return summaries

# Calculate the Gaussian probability distribution function for x
def calculate_probability(x, mean, stdev):
    exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent

# Calculate the probabilities of predicting each class for a given row
def calculate_class_probabilities(summaries, row):
    total_rows = sum([summaries[label][0][2] for label in summaries])
    probabilities = dict()
    for class_value, class_summaries in summaries.items():
        probabilities[class_value] = summaries[class_value][0][2]/float(total_rows)
        for i in range(len(class_summaries)):
            mean, stdev, _ = class_summaries[i]
            probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
    return probabilities
def predict(summaries, row):
    probabilities = calculate_class_probabilities(summaries, row)
    best_label, best_prob = None, -1
    for class_value, probability in probabilities.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value
    return best_label
 
# Naive Bayes Algorithm
def naive_bayes(train, test):
    summarize = summarize_by_class(train)
    predictions = list()
    for row in test:
        output = predict(summarize, row)
        predictions.append(output)
    return(predictions)
 

# Test calculating class probabilities
dataset = [[3.393533211,2.331273381,0],
           [3.110073483,1.781539638,0],
           [1.343808831,3.368360954,0],
           [3.582294042,4.67917911,0],
           [2.280362439,2.866990263,0],
           [7.423436942,4.696522875,1],
           [5.745051997,3.533989803,1],
           [9.172168622,2.511101045,1],
           [7.792783481,3.424088941,1],
           [7.939820817,0.791637231,1]]
summaries = summarize_by_class(dataset)
probabilities = calculate_class_probabilities(summaries, dataset[0])
print(probabilities)

{0: 0.05032427673372076, 1: 0.00011557718379945765}


In [42]:
import pandas as pd
import numpy as np

ftable = pd.read_csv('ftable.csv')
ftable = ftable[ftable['word'] != 'sad']
ftable = ftable[ftable['word'] != 'sad,']
ftable = ftable[ftable['word'] != ':(']
ftable = ftable[ftable['word'] != 'fun']
ftable = ftable[ftable['word'] != 'happy,']
ftable = ftable[ftable['word'] != 'happy']
ftable = ftable.drop_duplicates(subset = 'word')

#test = 'corona is not leaving to know what to do anymore'
#got this number from dataset creator
positive_instance = 617
negative_instance = 265.0
pc =0 
nc =0
for row in preprocessedTestSet:
    
    test_words = row[0]

    prob_positive = float(positive_instance/(positive_instance+negative_instance))
    prob_negative = 1 - prob_positive

    pos_word = 1.0*prob_positive
    neg_word = 1.0*prob_negative
    for i in range(len(test_words)):
        word = test_words[i]
        #print(word)
        index_val = ftable.index[ftable['word'] == word]
        if (len(index_val) > 0):
            #print(index_val[0])
            pos_val = ftable['positive'].iloc[index_val[0]]
            neg_val = ftable['negative'].iloc[index_val[0]]
            pos_word = pos_word * pos_val/positive_instance
            neg_word = neg_word * neg_val/negative_instance

    if pos_word > neg_word:
        pc+=1

    else:
        nc+=1
if(nc > pc):
    print("Overall tweet analysis negative with percentage:" +str(nc/100))
else:
    print("Overall tweet analysis positive with percentage:" +str(pc/100))



Overall tweet analysis negative with percentage:0.73
